https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data/data

In [ ]:
pip install kagglehub #библиотека для скачивания с каггл

In [ ]:
#скачаем последнюю версию с kaggle
import kagglehub

path = kagglehub.dataset_download("uciml/breast-cancer-wisconsin-data")

print("Path to dataset files:", path)

100%|██████████| 48.6k/48.6k [00:00<00:00, 519kB/s]

Extracting files...
Path to dataset files: C:\Users\ruzho\.cache\kagglehub\datasets\uciml\breast-cancer-wisconsin-data\versions\2


## О наборе данных

Параметры вычисляются на основе оцифрованного изображения тонкоигольной аспирационной биопсии (FNA) образования в молочной железе. Они описывают характеристики клеточных ядер, присутствующих на изображении.

Атрибутивная информация:

* 1)  ID number - Идентификационный номер
* 2)  Diagnosis - Диагноз (M = злокачественный, B = доброкачественный)/(M = malignant, B = benign)
* 3)

* Для каждого клеточного ядра вычисляются десять вещественных признаков:

* а) радиус (среднее значение расстояний от центра до точек на периметре)
* б) текстура (стандартное отклонение значений в градациях серого)
* в) периметр
* г) площадь
* д) гладкость (локальные изменения длины радиуса)
* е) компактность (периметр^2 / площадь - 1,0)
* ж) вогнутость (выраженность вогнутых участков контура)
* з) вогнутые точки (количество вогнутых участков контура)
* и) симметрия
* к) фрактальная размерность («приближение береговой линии» - 1)

Для каждого изображения были вычислены среднее значение, стандартная ошибка и «наихудшее» или наибольшее (среднее значение из трёх
наибольших значений) из этих характеристик,
в результате чего получилось 30 характеристик. Например, поле 3 — это средний радиус, поле
13 — стандартная ошибка радиуса, поле 23 — наихудший радиус.

Все значения признаков перекодированы четырьмя значащими цифрами.

Отсутствующие значения атрибутов: отсутствуют

Распределение по классам: 357 доброкачественных, 212 злокачественных

https://minds.wisconsin.edu/bitstream/handle/1793/59692/TR1131.pdf;jsessionid=944B13EC1D0D6DCB528D22A999948C8E?sequence=1
* ссылка на статью 
## Извлечение ядерных признаков для диагностики опухоли молочной железы
У. Стрит, У. Вольберг, О. Мангасарян
Опубликовано в журнале «Электронная визуализация» 29 июля 1993 г.
Медицина

### Аннотация
Для создания высокоточной системы диагностики опухолей молочной железы были использованы интерактивные методы обработки изображений, а также индуктивный классификатор на основе линейного программирования. Небольшая часть препарата, полученного с помощью тонкоигольной аспирационной биопсии, отбирается и оцифровывается. Это позволяет проводить точный автоматизированный анализ размера, формы и текстуры ядер. Для каждого ядра вычисляются десять таких характеристик, и для диапазона изолированных клеток определяются среднее значение, наибольшее (или «наихудшее») значение и стандартная ошибка каждой характеристики. После анализа таким образом 569 изображений были протестированы различные комбинации характеристик, чтобы найти те, которые лучше всего отделяют доброкачественные образцы от злокачественных. Десятикратная точность перекрестной проверки в 97% была достигнута с использованием одной разделительной плоскости по трем из тридцати признаков: средней текстуре, наихудшей площади и наихудшей гладкости. Это представляет собой улучшение по сравнению с лучшими результатами диагностики в медицинской литературе. В настоящее время система используется в больницах Университета Висконсина. Тот же набор признаков также использовался в гораздо более сложной задаче прогнозирования отдаленного рецидива злокачественной опухоли у пациентов, что привело к точности 86%.

In [2]:
import pandas as pd                  # Импорт библиотеки pandas для работы с табличными данными (DataFrame)
import numpy as np                   # Импорт библиотеки numpy для работы с массивами и числовыми операциями
import seaborn as sns                # Импорт библиотеки seaborn для создания красивых и информативных визуализаций данных
from matplotlib import pyplot as plt # Импорт функции pyplot из библиотеки matplotlib для создания графиков и визуализаций

from sklearn.preprocessing import OneHotEncoder, StandardScaler # Импорт классов для кодирования категориальных признаков и стандартизации данных
from sklearn.model_selection import train_test_split            # Импорт функций для разбиения данных на тренировочную и тестовую выборки

from sklearn.linear_model import LogisticRegression    # Импорт модели логистической регрессии
from sklearn.neighbors import KNeighborsClassifier     # Импорт классификатора K-ближайших соседей
from sklearn.tree import DecisionTreeClassifier        # Импорт классификатора на основе дерева решений
from sklearn.svm import SVC                            # Импорт классификатора на основе метода опорных векторов (SVM)
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

from sklearn import metrics      # Импорт всего модуля metrics для работы с метриками оценки моделей
from sklearn.metrics import (    # Импорт конкретных функций оценки из подмодуля metrics
    accuracy_score,              # Импорт функции для оценки точности (accuracy) модели, т.е. доли правильно предсказанных примеров
    balanced_accuracy_score,     # Импорт функции для оценки сбалансированной точности, учитывающей классовую неоднородность
    precision_score,             # Импорт функции для вычисления точности (precision)
    recall_score,                # Импорт функции для вычисления полноты (recall)
    confusion_matrix,            # Импорт функции для создания матрицы ошибок (confusion matrix):TN, TP, FN, FP
    multilabel_confusion_matrix, # Импорт функции для создания матрицы ошибок для случаев с многомерной классификацией
    f1_score,                    # Импорт функции для вычисления F1-меры
    roc_auc_score,               # Импорт функции для вычисления AUC-ROC (площадь под кривой приемлемости и ошибок)
    roc_curve,                   # Импорт функции для вычисления координат для построения ROC-кривой
    average_precision_score,     # Импорт функции для вычисления средней точности (average precision)
    precision_recall_curve       # Импорт функции для вычисления и визуализации кривой "точность-полнота" (precision-recall curve)
)